# MATH7016 Deep Learning
### Coursework 2025

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy.random import seed
import tensorflow as tf
from tensorflow import keras
from keras import backend as K

from sklearn.model_selection import KFold
from keras.layers import Activation
from keras.utils import get_custom_objects
from sklearn.preprocessing import StandardScaler

tf.random.set_seed(42)

In [2]:
# Read training and test datasets
train_df = pd.read_csv('DL-2025-CW-data/Data_A.csv')
test_df = pd.read_csv('DL-2025-CW-data/Data_B_nolabels.csv')

# Define column names
columns_names = ['target']
level_cols=[]
for i in range(4):
    level_cols +=[f'ask_price{i+1}',f'ask_size{i+1}',f'bid_price{i+1}',f'bid_size{i+1}']
change_cols =  []
for i in range(5):
    change_cols+=[f'midprice_change{i+1}']
columns_names+=level_cols+change_cols
train_df.columns = columns_names
test_df.columns = columns_names[1:]

test_df.head()

,ask_price1,ask_size1,bid_price1,bid_size1,ask_price2,ask_size2,bid_price2,bid_size2,ask_price3,ask_size3,...,bid_size3,ask_price4,ask_size4,bid_price4,bid_size4,midprice_change1,midprice_change2,midprice_change3,midprice_change4,midprice_change5
0,696400.0,16,696000.0,12,696500.0,57,695900.0,118,696600.0,100,...,262,696700.0,150,695700.0,104,1,0,1,0,1
1,740800.0,2,740400.0,20,741000.0,60,740200.0,27,741200.0,156,...,31,741300.0,200,740000.0,170,0,1,0,0,1
2,730900.0,1,730200.0,230,731000.0,111,730100.0,86,731100.0,42,...,136,731200.0,100,729900.0,132,1,1,0,0,1
3,630600.0,100,630300.0,69,630700.0,110,630200.0,2,630800.0,219,...,1,630900.0,101,630000.0,104,0,1,0,0,0
4,851100.0,579,850300.0,25,851200.0,17,850100.0,287,851400.0,307,...,270,851500.0,223,849900.0,72,1,0,1,0,1


In [3]:
def mid_price(df,  bid_p, ask_p):
    return (df[bid_p] + df[ask_p])/2

def calc_wap(df, bid_p, ask_p, bid_s, ask_s):
    return (df[bid_p] * df[ask_s] + df[ask_p] * df[bid_s]) / (df[bid_s] + df[ask_s])


In [4]:
def preprocessor(df):
    # Step 1: Calculate price & volume features
    for i in range(1,5):
        df[f"wap{i}"] = calc_wap(df, f"bid_price{i}", f"ask_price{i}", f"bid_size{i}", f"ask_size{i}")
        #df[f"wmp{i}"] = calc_wmp(df, f"bid_price{i}", f"ask_price{i}", f"bid_size{i}", f"ask_size{i}")
        df[f"price_spread{i}"] = (df[f"ask_price{i}"] - df[f"bid_price{i}"]) / ((df[f"ask_price{i}"] + df[f"bid_price{i}"]) / 2)
        df[f'midprice{i}']=mid_price(df,f'bid_price{i}',f'ask_price{i}')
 
        df[f"order_imbalance{i}"] = (df[f"bid_size{i}"] - df[f"ask_size{i}"]) / (df[f"bid_size{i}"] + df[f"ask_size{i}"])
        df[f'bid_ask_ratio{i}'] = df[f'bid_size{i}'] / (df[f'bid_size{i}'] + df[f'ask_size{i}'])
        
    # Price features

    df['bid_depth_ratio'] = df['bid_size1'] / (df[['bid_size1','bid_size2','bid_size3','bid_size4']].sum(axis=1))
    df['ask_depth_ratio'] = df['ask_size1'] / (df[['ask_size1','ask_size2','ask_size3','ask_size4']].sum(axis=1))

    # Volume features
    df["total_volume"] = df[["ask_size1", "ask_size2", "bid_size1", "bid_size2"]].sum(axis=1)
    df["volume_imbalance"] = (
        (df["ask_size1"] + df["ask_size2"]) - (df["bid_size1"] + df["bid_size2"])
    ).abs()

    # Order imbalances 
    df['bid_vol_ratio'] = df[['bid_size1','bid_size2']].sum(axis=1) / df[['bid_size1','bid_size2','ask_size1','ask_size2']].sum(axis=1)
    df['ask_vol_ratio'] = 1 - df['bid_vol_ratio']
    
    # Step 3: Drop raw order book columns
    #df = df.drop(columns=level_cols)
    
    return df


In [5]:
train_nn = preprocessor(train_df)
test_nn = preprocessor(test_df)


In [6]:
def swish(x, beta = 1):
    return (x * K.sigmoid(beta * x))

get_custom_objects().update({'swish': Activation(swish)})

hidden_units = (128,64,32)
feature_cols = [c for c in train_nn.columns if c != 'target']
num_features = len(feature_cols)

def base_model():
  
    num_input = keras.Input(shape=(num_features), name='num_data')
    x = keras.layers.Concatenate()([num_input])
   
    for n_hidden in hidden_units:
        x = keras.layers.Dense(n_hidden, activation='swish')(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dropout(0.3)(x)
    out = keras.layers.Dense(1, activation='sigmoid', name='prediction')(x)

    model = keras.Model(
        inputs=[num_input],
        outputs=out
    )
    return model


In [7]:
train_nn = train_nn.fillna(0)
test_nn = test_nn.fillna(0)

X = train_nn[feature_cols]
y = train_nn['target']

test_predictions_nn = np.zeros(test_nn.shape[0])
scores_folds = {'NN_model': []}

# KFold 
kfolds = 5
kf = KFold(n_splits=kfolds, shuffle=True, random_state=42)

counter = 1

for train_index, val_index in kf.split(X):

    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_val = X.iloc[val_index]
    y_val = y.iloc[val_index]
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)

    model = base_model()
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.005),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # ---- callbacks ----
    # Early stopping callback
    es = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=20, verbose=0,
        mode='min', restore_best_weights=True)

    # Learning rate reduction callback 
    plateau = tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.2, patience=7, verbose=0,
        mode='min')
    model.fit(
        [X_train_scaled],
        y_train,
        batch_size=512,
        epochs=200,
        validation_data=([X_val_scaled], y_val),
        callbacks=[es, plateau],
        shuffle=True,
        verbose=1
    )
    
    preds = model.predict([X_val_scaled]).reshape(-1)
    score = np.mean((preds.round() != y_val.values).astype(float)) 
    print(f'Fold {counter}: Accuracy = {1 - score:.5f}')
    scores_folds['NN_model'].append(1 - score)
    
    X_test = test_nn[feature_cols]
    X_test_scaled = scaler.transform(X_test)

    test_predictions_nn += model.predict([X_test_scaled]).reshape(-1) / kfolds
   
    counter += 1


print("CV folds scores:", scores_folds['NN_model'])

Epoch 1/200
313/313 [==============================] - 1s 3ms/step - loss: 0.5850 - accuracy: 0.6820 - val_loss: 0.5525 - val_accuracy: 0.6985 - lr: 0.0050
Epoch 2/200
313/313 [==============================] - 1s 3ms/step - loss: 0.5594 - accuracy: 0.6930 - val_loss: 0.5441 - val_accuracy: 0.6995 - lr: 0.0050
Epoch 3/200
313/313 [==============================] - 1s 3ms/step - loss: 0.5520 - accuracy: 0.6957 - val_loss: 0.5355 - val_accuracy: 0.7047 - lr: 0.0050
Epoch 4/200
313/313 [==============================] - 1s 3ms/step - loss: 0.5476 - accuracy: 0.6993 - val_loss: 0.5365 - val_accuracy: 0.7063 - lr: 0.0050
Epoch 5/200
313/313 [==============================] - 1s 3ms/step - loss: 0.5437 - accuracy: 0.7016 - val_loss: 0.5297 - val_accuracy: 0.7085 - lr: 0.0050
Epoch 6/200
313/313 [==============================] - 1s 3ms/step - loss: 0.5420 - accuracy: 0.7042 - val_loss: 0.5279 - val_accuracy: 0.7114 - lr: 0.0050
Epoch 7/200
313/313 [==============================] - 1s 3ms/st

Epoch 1/200
313/313 [==============================] - 1s 3ms/step - loss: 0.5834 - accuracy: 0.6834 - val_loss: 0.5574 - val_accuracy: 0.6981 - lr: 0.0050
Epoch 2/200
313/313 [==============================] - 1s 3ms/step - loss: 0.5606 - accuracy: 0.6930 - val_loss: 0.5445 - val_accuracy: 0.6998 - lr: 0.0050
Epoch 3/200
313/313 [==============================] - 1s 3ms/step - loss: 0.5536 - accuracy: 0.6949 - val_loss: 0.5366 - val_accuracy: 0.7053 - lr: 0.0050
Epoch 4/200
313/313 [==============================] - 1s 3ms/step - loss: 0.5485 - accuracy: 0.6994 - val_loss: 0.5318 - val_accuracy: 0.7084 - lr: 0.0050
Epoch 5/200
313/313 [==============================] - 1s 3ms/step - loss: 0.5449 - accuracy: 0.7022 - val_loss: 0.5289 - val_accuracy: 0.7097 - lr: 0.0050
Epoch 6/200
313/313 [==============================] - 1s 3ms/step - loss: 0.5424 - accuracy: 0.7028 - val_loss: 0.5292 - val_accuracy: 0.7102 - lr: 0.0050
Epoch 7/200
313/313 [==============================] - 1s 3ms/st

Epoch 1/200
313/313 [==============================] - 3s 7ms/step - loss: 0.5836 - accuracy: 0.6828 - val_loss: 0.5566 - val_accuracy: 0.6938 - lr: 0.0050
Epoch 2/200
313/313 [==============================] - 2s 6ms/step - loss: 0.5581 - accuracy: 0.6931 - val_loss: 0.5450 - val_accuracy: 0.7000 - lr: 0.0050
Epoch 3/200
313/313 [==============================] - 2s 6ms/step - loss: 0.5503 - accuracy: 0.6963 - val_loss: 0.5385 - val_accuracy: 0.7028 - lr: 0.0050
Epoch 4/200
313/313 [==============================] - 2s 6ms/step - loss: 0.5456 - accuracy: 0.7012 - val_loss: 0.5352 - val_accuracy: 0.7063 - lr: 0.0050
Epoch 5/200
313/313 [==============================] - 2s 6ms/step - loss: 0.5423 - accuracy: 0.7036 - val_loss: 0.5349 - val_accuracy: 0.7053 - lr: 0.0050
Epoch 6/200
313/313 [==============================] - 2s 6ms/step - loss: 0.5403 - accuracy: 0.7048 - val_loss: 0.5304 - val_accuracy: 0.7099 - lr: 0.0050
Epoch 7/200
313/313 [==============================] - 2s 6ms/st

KeyboardInterrupt: 

In [ ]:
final_pred = (test_predictions_nn > 0.5).astype(int)
#np.savetxt("02205857_wang.txt", final_pred, fmt="%d")